In [122]:
import pandas as pd
from pymongo import MongoClient
import re
import numpy as np
from Functions.location import *
from Functions.money import moneyClean

In [123]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

#### Here I make a Pymongo query to get just the companies with certain conditions ####

In [124]:
companies = db.companies.find({
    "offices":{
        "$not":{
            "$size":0
        }
    }
, "founded_year":{
    "$gte": 2007
}
}
,{"name":1, "founded_year":1, "offices":1, "category_code":1, "total_money_raised":1, "deadpooled_year":1, "number_of_employees":1})


#### Create the dataframe ####

In [125]:
clean_companies = pd.DataFrame(companies)
clean_companies.head()

,_id,name,category_code,number_of_employees,founded_year,deadpooled_year,total_money_raised,offices
0,52cdef7c4bab8bd675297d97,Scribd,news,50.0,2007,NaN,$25.8M,"[{'description': 'HQ', 'address1': '539 Bryant..."
1,52cdef7c4bab8bd675297d9c,MeetMoi,social,15.0,2007,NaN,$5.58M,"[{'description': None, 'address1': '', 'addres..."
2,52cdef7c4bab8bd675297da0,Babelgum,games_video,NaN,2007,2013.0,$13.2M,"[{'description': '', 'address1': '', 'address2..."
3,52cdef7c4bab8bd675297daa,Sparter,games_video,NaN,2007,2008.0,$0,"[{'description': None, 'address1': None, 'addr..."
4,52cdef7c4bab8bd675297da9,Mahalo,web,40.0,2007,NaN,$21M,"[{'description': '', 'address1': '3525 Eastham..."


#### Apply Location Function ####

In [126]:
geo_office = clean_companies[["offices"]].apply(getGeoLocation, result_type="expand", axis=1) #Aplico la función

#### Join data to create Principal Office ####

In [127]:
df_clean = pd.concat([clean_companies,geo_office], axis=1)[["principal_office"]]

In [128]:
clean_companies['geoDescription'] = df_clean
clean_companies.head()

,_id,name,category_code,number_of_employees,founded_year,deadpooled_year,total_money_raised,offices,geoDescription
0,52cdef7c4bab8bd675297d97,Scribd,news,50.0,2007,NaN,$25.8M,"[{'description': 'HQ', 'address1': '539 Bryant...","{'type': 'Point', 'coordinates': [-122.404052,..."
1,52cdef7c4bab8bd675297d9c,MeetMoi,social,15.0,2007,NaN,$5.58M,"[{'description': None, 'address1': '', 'addres...","{'type': 'Point', 'coordinates': [-73.985506, ..."
2,52cdef7c4bab8bd675297da0,Babelgum,games_video,NaN,2007,2013.0,$13.2M,"[{'description': '', 'address1': '', 'address2...","{'type': 'Point', 'coordinates': [-6.267494, 5..."
3,52cdef7c4bab8bd675297daa,Sparter,games_video,NaN,2007,2008.0,$0,"[{'description': None, 'address1': None, 'addr...","{'type': 'Point', 'coordinates': [-95.712891, ..."
4,52cdef7c4bab8bd675297da9,Mahalo,web,40.0,2007,NaN,$21M,"[{'description': '', 'address1': '3525 Eastham...","{'type': 'Point', 'coordinates': [-118.487267,..."


In [129]:
clean_companies['category_code'].value_counts()

web                 1167
software             514
games_video          365
advertising          287
mobile               277
other                222
ecommerce            222
enterprise           213
network_hosting      160
public_relations     154
consulting           141
search               115
cleantech             61
biotech               53
hardware              37
social                29
analytics             29
security              24
education             19
news                  18
music                 18
finance               16
photo_video           14
travel                14
messaging             12
sports                 8
health                 8
legal                  6
semiconductor          6
medical                6
fashion                5
hospitality            4
real_estate            4
design                 3
manufacturing          3
nonprofit              1
nanotech               1
automotive             1
transportation         1
Name: category_code, dtyp

#### Just Tech Business ####

In [130]:
list_category = {'web': 'tech', 'software': 'tech', 
                 'games_video':'tech', 'advertising':'tech', 'mobile':'tech', 'ecommerce':'tech', 'search':'tech', 'network_hosting':'tech'
                ,'consulting':'tech','hardware':'tech', 'biotech':'tech', 'cleantech':'tech', 'analytics':'tech','hardware':'tech', 'photo_video':'tech',
                'messaging':'tech', 'design':'tech'}

In [131]:
clean_companies = clean_companies.replace(to_replace=list_category, inplace=False)

#### Removing Null Values ####

In [132]:
clean_companies['founded_year'] = clean_companies['founded_year'].dropna(axis=0).astype('int64', inplace=True)

#### Create Longitude and Latitude Columns ####

In [133]:
latitude = [d[0].get('latitude') for d in clean_companies.offices]
longitude = [d[0].get('longitude') for d in clean_companies.offices]

In [134]:
clean_companies['latitude'] = latitude
clean_companies['longitude'] = longitude

#### City and Country in two Columns ####

In [135]:
city = [d[0].get('city') for d in clean_companies.offices]
country = [d[0].get('country_code') for d in clean_companies.offices]

In [136]:
clean_companies['city'] = city
clean_companies['country'] = country
clean_companies.head()

,_id,name,category_code,number_of_employees,founded_year,deadpooled_year,total_money_raised,offices,geoDescription,latitude,longitude,city,country
0,52cdef7c4bab8bd675297d97,Scribd,news,50.0,2007,NaN,$25.8M,"[{'description': 'HQ', 'address1': '539 Bryant...","{'type': 'Point', 'coordinates': [-122.404052,...",37.789634,-122.404052,San Francisco,USA
1,52cdef7c4bab8bd675297d9c,MeetMoi,social,15.0,2007,NaN,$5.58M,"[{'description': None, 'address1': '', 'addres...","{'type': 'Point', 'coordinates': [-73.985506, ...",40.757929,-73.985506,New York City,USA
2,52cdef7c4bab8bd675297da0,Babelgum,tech,NaN,2007,2013.0,$13.2M,"[{'description': '', 'address1': '', 'address2...","{'type': 'Point', 'coordinates': [-6.267494, 5...",53.344104,-6.267494,London,GBR
3,52cdef7c4bab8bd675297daa,Sparter,tech,NaN,2007,2008.0,$0,"[{'description': None, 'address1': None, 'addr...","{'type': 'Point', 'coordinates': [-95.712891, ...",37.090240,-95.712891,None,USA
4,52cdef7c4bab8bd675297da9,Mahalo,tech,40.0,2007,NaN,$21M,"[{'description': '', 'address1': '3525 Eastham...","{'type': 'Point', 'coordinates': [-118.487267,...",34.017606,-118.487267,Culver City,USA


#### Deleting useless Columns ####

In [137]:
def dropcolumns(data, columns):
        data = data.drop(columns, axis=1)
        return data

In [138]:
clean_companies = dropcolumns(clean_companies, ['offices', '_id'])

#### Cleaning Deadpooleds ####

In [139]:
clean_companies = clean_companies[clean_companies['deadpooled_year'].isna()] 


#### Apply function to clean some locations ####

In [90]:
clean_companies['city'] = clean_companies['city'].apply(cleanLocation)


#### Final Cleaning Columns ####

In [91]:
clean_companies.dropna(subset=['founded_year', 'latitude', 'longitude'], inplace=True)

In [92]:
clean_companies = dropcolumns(clean_companies, 'deadpooled_year')

#### Apply function to moneyClean ####

In [141]:
clean_companies['total_money_raised'] = clean_companies['total_money_raised'].apply(moneyClean)

#### Fill Null Values ####

In [94]:
def fillNaN(data, col):
    [data[col].fillna(0, inplace=True) for col in data.columns]
    return data

In [95]:
clean_companies = fillNaN(clean_companies, 'number_of_employees')

#### Just Business with employees, money raised and in the USA ####

In [96]:
clean_companies = clean_companies[clean_companies['number_of_employees'] != 0.0]
clean_companies = clean_companies[clean_companies['total_money_raised']!=0]

In [97]:
clean_companies = clean_companies[clean_companies['country'] != 'USA']

#### Outputs ####

In [98]:
clean_companies = clean_companies.reset_index(drop=True)

clean_companies.head()

,name,category_code,number_of_employees,founded_year,total_money_raised,geoDescription,latitude,longitude,city,country
0,Wakoopa,tech,8.0,2007,1000000,"{'type': 'Point', 'coordinates': [4.8948623, 5...",52.374523,4.894862,Amsterdam,NLD
1,boo-box,tech,50.0,2007,300,"{'type': 'Point', 'coordinates': [-46.679287, ...",-23.558584,-46.679287,Sao Paulo,BRA
2,Seedcamp,finance,4.0,2007,5000000,"{'type': 'Point', 'coordinates': [-0.1418973, ...",51.510880,-0.141897,London,GBR
3,Flixwagon,tech,10.0,2007,2500000,"{'type': 'Point', 'coordinates': [34.7595, 32....",32.055400,34.759500,Tel-Aviv,ISR
4,UnLtdWorld,enterprise,4.0,2007,199,"{'type': 'Point', 'coordinates': [-0.0926903, ...",51.522413,-0.092690,London,GBR


In [99]:
clean_companies.to_json('./Output/clean_companies.json', orient='records')

In [100]:
clean_companies.to_csv('./Output/clean_companies.csv', index=True)